In [1]:
import prefect
from prefect import task, Flow, Parameter, Client
from prefect.run_configs import KubernetesRun
from prefect.schedules import IntervalSchedule
from prefect.storage import S3

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

from datetime import timedelta
import time
import getpass

import numpy as np
import pandas as pd

import mlflow
import requests

# Run the model in prefect

In [22]:
@task
def fetch_data():
    # Fetch data from the internet
    csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    data = pd.read_csv(csv_url, sep=";")
    return data

@task
def train_model(data, alpha=0.5, l1_ratio=0.5):
    # Train a regression model
    train, test = train_test_split(data)
    
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    # Evaluate the model with the helper function
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    
def eval_metrics(actual, pred):
    # Helper function to evalue the regression model
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [25]:
def get_prefect_token():
    # Helper function to retrieve prefect token
    r = requests.get(auth_url)
    jsn = r.json()
    action_url = jsn["ui"]["action"]
    data = {"password_identifier": username, "password": password, "method": "password"}
    headers = {"Accept": "application/json", "Content-Type": "application/json"}
    r = requests.post(action_url, json=data, headers=headers)
    r.raise_for_status()
    jsn = r.json()
    session_token = jsn["session_token"]
    return session_token

def get_tenant_id():
    # Helper function to retrieve tentant id
    session_token = get_prefect_token()
    prefect_client = Client(api_server=prefect_url, api_key=get_prefect_token())
    return prefect_client.get_available_tenants()[0]["id"]

In [30]:
def create_prefect_flow():
    # Execution environment
    run_config = KubernetesRun(
        labels=["dev"],
        service_account_name="prefect-server-serviceaccount",
        image=docker_image
    )
    storage = S3(s3_bucket)

    session_token = get_prefect_token()
    prefect_client = Client(api_server=prefect_url, api_key=session_token, tenant_id=get_tenant_id())

    # Define the flow
    with Flow("train-wine-quality-model", storage=storage, run_config=run_config) as flow:
        data = fetch_data()
        train_model(data=data, alpha=0.3, l1_ratio=0.3)

    # Deploy and trigger
    prefect_client.create_project(project_name=prefect_project_name)
    training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)
    flow_run_id = prefect_client.create_flow_run(flow_id=training_flow_id, run_name=f"run {prefect_project_name}")


s3_bucket = "demo4nic-mlflow2"
domain = "mlops2.demo4nic.com"
prefect_project_name = "wine-quality-project"         # you can use what you want here
docker_image = "smandaric/prefect:winequality-v1"    # any docker image that has the required Python dependencies
prefect_url = f"https://prefect.{domain}/graphql"
auth_url = f"https://{domain}/.ory/kratos/public/self-service/login/api"
username = "nicdemo1@alchemyst.xyz"
password = getpass.getpass()


create_prefect_flow()

 ················


<Response [200]>
<Response [200]>
<Response [200]>
[2022-05-31 17:12:33+0200] INFO - prefect.S3 | Uploading train-wine-quality-model/2022-05-31t15-12-33-305617-00-00 to demo4nic-mlflow2


/tmp/ipykernel_296/3978813208.py:20: UserWarning: No result handler was specified on your Flow. Cloud features such as input caching and resuming task runs from failure may not work properly.
  training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)


Flow URL: https://cloud.prefect.io/default/flow/71dc55b1-755f-4680-b463-24676eeaa6b0
 └── ID: 77654a95-007f-4df7-adf3-9a614c2f1d1f
 └── Project: wine-quality-project
 └── Labels: ['dev']
